# 数据生成二进制文件

In [22]:
!mkdir data/IWSLT14 data/IWSLT2014/de_en data/IWSLT2014/de_en/raw

In [5]:
!rm -rf data

In [1]:
!rm -rf ../data/IWSLT14/de_en/baseline ../data/IWSLT15/zh_en/cross_enc

In [6]:
!rm -rf ../data/IWSLT15/zh_en/model/20210107context/checkpoints/* ../data/IWSLT15/zh_en/model/20210107context/tensorboard/* 

In [2]:
!rm ../data/IWSLT15/zh_en/model/20210107context/processed/*

In [4]:
!python preprocess.py --source-lang zh --target-lang en\
--trainpref ../data/IWSLT15/zh_en/bpe/train.bpe.context \
--validpref ../data/IWSLT15/zh_en/bpe/dev.bpe.context \
--testpref ../data/IWSLT15/zh_en/bpe/test.bpe.context \
--criterion=label_smoothed_cross_entropy \
--destdir ../data/IWSLT15/zh_en/model/20210109context/processed_context

2021-01-09 17:07:10 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='label_smoothed_cross_entropy', dataset_impl='mmap', destdir='../data/IWSLT15/zh_en/model/20210109context/processed_context', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='zh', srcdict=None, target_lang='en', task='translation', tensorboard_logdir=None, testpref='../data/IWSLT15/zh_en/bpe/test.bpe.context', tgtdict=None, 

# 训练模型

In [ ]:
!CUDA_VISIBLE_DEVICES=0 nohup python train.py ../data/IWSLT15/zh_en/model/20210109context/processed \
--tensorboard-logdir='../data/IWSLT15/zh_en/model/20210109context/tensorboard' \
--save-dir='../data/IWSLT15/zh_en/model/20210109context/checkpoints' \
--source-lang=zh --target-lang=en \
--arch=context \
--max-tokens 8000 \
--left-pad-source False \
--reset-optimizer \
--save-interval=1 --no-progress-bar --keep-last-epochs=5 --keep-best-checkpoints=5 \
--attention-dropout=0.3 --criterion=label_smoothed_cross_entropy \
--lr-scheduler=inverse_sqrt --min-lr 1e-09 --label-smoothing=0.1 --update-freq=2 \
--optimizer adam --adam-betas '(0.9, 0.98)' --warmup-init-lr 1e-07 \
--lr 0.0005 --warmup-updates 8000 --dropout 0.1 --weight-decay 0.0001 \
--valid-subset=valid --max-epoch=50 \
--eval-bleu --eval-bleu-remove-bpe \
--skip-invalid-size-inputs-valid-test \
--best-checkpoint-metric 'bleu' --maximize-best-checkpoint-metric \
--ddp-backend=no_c10d >../data/IWSLT15/zh_en/model/20210109context/log.txt  2>&1 &

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train.py ../data/IWSLT15/zh_en/model/20210109context/processed_context \
--tensorboard-logdir='../data/IWSLT15/zh_en/model/20210109context/tensorboard' \
--save-dir='../data/IWSLT15/zh_en/model/20210109context/checkpoints' \
--source-lang=zh --target-lang=en \
--arch=context \
--max-tokens 8000 \
--left-pad-source False \
--reset-optimizer \
--save-interval=1 --no-progress-bar --keep-last-epochs=5 --keep-best-checkpoints=5 \
--attention-dropout=0.3 --criterion=label_smoothed_cross_entropy \
--lr-scheduler=inverse_sqrt --min-lr 1e-09 --label-smoothing=0.1 --update-freq=2 \
--optimizer adam --adam-betas '(0.9, 0.98)' --warmup-init-lr 1e-07 \
--lr 0.0005 --warmup-updates 8000 --dropout 0.1 --weight-decay 0.0001 \
--valid-subset=valid --max-epoch=100 \
--eval-bleu --eval-bleu-remove-bpe \
--skip-invalid-size-inputs-valid-test \
--best-checkpoint-metric 'bleu' --maximize-best-checkpoint-metric \
--ddp-backend=no_c10d \

2021-01-09 17:09:28 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='context', attention_dropout=0.3, batch_size=None, batch_size_valid=None, best_checkpoint_metric='bleu', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='../data/IWSLT15/zh_en/model/20210109context/processed_context', data_buffer_size=10, dataset_impl=None, ddp_backend='no_c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=2048, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, de

2021-01-09 17:09:29 | INFO | fairseq.trainer | detected shared parameter: encoder.embed_tokens.weight <- encoder.context_encoder.embed_tokens.weight
2021-01-09 17:09:29 | INFO | fairseq.utils | ***********************CUDA enviroments for all 1 workers***********************
2021-01-09 17:09:29 | INFO | fairseq.utils | rank   0: capabilities =  7.5  ; total memory = 47.459 GB ; name = Quadro RTX 8000                         
2021-01-09 17:09:29 | INFO | fairseq.utils | ***********************CUDA enviroments for all 1 workers***********************
2021-01-09 17:09:29 | INFO | fairseq_cli.train | training on 1 devices (GPUs/TPUs)
2021-01-09 17:09:29 | INFO | fairseq_cli.train | max tokens per GPU = 8000 and max sentences per GPU = None
2021-01-09 17:09:30 | INFO | fairseq.trainer | loaded checkpoint ../data/IWSLT15/zh_en/model/20210109context/checkpoints/checkpoint_last.pt (epoch 39 @ 0 updates)
2021-01-09 17:09:30 | INFO | fairseq.trainer | NOTE: your device may support faster training

# 翻译

In [2]:
!CUDA_VISIBLE_DEVICES=0 python generate.py ../data/IWSLT15/zh_en/model/20210109context/processed \
--device-id 6 \
--path ../data/IWSLT15/zh_en/model/20210109context/checkpoints/checkpoint_last.pt \
--batch-size 200 --beam 5 \
--criterion=label_smoothed_cross_entropy \
--source-lang=zh --target-lang=en \
--skip-invalid-size-inputs-valid-test \
--remove-bpe \

2021-01-09 16:55:46 | INFO | fairseq_cli.generate | Namespace(all_gather_list_size=16384, batch_size=200, batch_size_valid=200, beam=5, bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', constraints=None, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='../data/IWSLT15/zh_en/model/20210109context/processed', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoding_format=None, device_id=6, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', diverse_beam_groups=-1, diverse_beam_strength=0.5, diversity_rate=-1.0, empty_cache_freq=0, eval_bleu=False, eval_bleu_args=None, eval_bleu_detok='space', eval_bleu_detok_args=None, eval_bleu_print_samples=False, eval_bleu_remove_bpe=None, eval_tokenized_bleu=False, fast_stat_sync=False, find_unused_pa

S-3256	这些 就是 神经 结构 。
T-3256	and those are the neural structures .
H-3256	-0.9207095503807068	these are the neurological structures .
D-3256	-0.9207095503807068	these are the neurological structures .
P-3256	-1.5440 -0.1561 -2.2996 -1.6607 -1.2162 -0.1337 -0.2676 -0.0878
S-3184	就 有点 像是 这样 。
T-3184	they sort of go like this .
H-3184	-0.6103111505508423	it ' s a bit like this .
D-3184	-0.6103111505508423	it ' s a bit like this .
P-3184	-1.0310 -0.5650 -0.1021 -1.8057 -1.0110 -0.1617 -0.4505 -0.2510 -0.1148
S-2892	每晚 都 努力 工作 。
T-2892	we worked well into each night .
H-2892	-1.0478506088256836	it ' s hard to work every night .
D-2892	-1.0478506088256836	it ' s hard to work every night .
P-2892	-2.9908 -0.7914 -0.0852 -2.8882 -1.5470 -0.2752 -1.2380 -0.2786 -0.2718 -0.1122
S-2813	一个 不断 延续 的 故事
T-2813	this is not a finished story .
H-2813	-1.027242660522461	it ' s a continuum .
D-2813	-1.027242660522461	it ' s a continuum .
P-2813	-2.7925 -0.2565 -0.0943 -0.8744 -1.9267 -0.8882 -1.2606 -0.124

S-2018	它 叫做 数字 云 。
T-2018	it ' s called the cloud .
H-2018	-0.2655421793460846	it ' s called the digital cloud .
D-2018	-0.2655421793460846	it ' s called the digital cloud .
P-2018	-0.2337 -0.1486 -0.0991 -0.1403 -0.6764 -0.0742 -0.7783 -0.1444 -0.0949
S-554	燃料 的 问题 解决 了
T-554	in terms of fuel , this really solves the problem .
H-554	-0.9530284404754639	the fuel problem is solved .
D-554	-0.9530284404754639	the fuel problem is solved .
P-554	-1.5000 -0.1609 -1.6281 -1.4159 -1.6117 -0.2487 -0.1060
S-1814	我们 没有 那样 的 情况
T-1814	we do n 't have that kind of situation .
H-1814	-0.6301849484443665	we do n 't have that .
D-1814	-0.6301849484443665	we do n 't have that .
P-1814	-0.7975 -1.1529 -0.1323 -0.0741 -0.3188 -0.7452 -1.7025 -0.1182
S-2520	你 看到 所有 这些 博物馆
T-2520	you look around at all these museums here .
H-2520	-0.4510726034641266	you see all these museums .
D-2520	-0.4510726034641266	you see all these museums .
P-2520	-0.8516 -0.8490 -0.2538 -1.1661 -0.1486 -0.0447 -0.2013 -0.0935
S-28

S-787	我 是 个 记者 ，
T-787	i mean , i ' m a journalist .
H-787	-0.35675469040870667	i ' m a journalist .
D-787	-0.35675469040870667	i ' m a journalist .
P-787	-0.3681 -1.0997 -0.0839 -0.2040 -0.4156 -0.2059 -0.1202
S-3465	我谈 了 很多 问题
T-3465	i talk a lot about these issues .
H-3465	-0.7073063850402832	i ' ve talked a lot about that .
D-3465	-0.7073063850402832	i ' ve talked a lot about that .
P-3465	-1.1109 -1.0617 -0.5685 -0.9256 -0.3507 -0.2014 -0.2118 -2.2610 -0.2721 -0.1094
S-1145	情感 该 如何 测量 ？
T-1145	what are the emotions that can be measured ?
H-1145	-0.7176251411437988	how do emotions measure ?
D-1145	-0.7176251411437988	how do emotions measure ?
P-1145	-0.6208 -0.8156 -2.5494 -0.1919 -0.0252 -0.1028
S-2151	这是 一幅 循环 的 图片
T-2151	that ' s a painting of a circle .
H-2151	-0.7570780515670776	this is a picture of a cycle .
D-2151	-0.7570780515670776	this is a picture of a cycle .
P-2151	-1.1146 -0.1927 -0.2453 -2.0530 -0.7427 -0.3622 -1.7820 -0.2219 -0.0993
S-2561	你 放大 ， 真的 放大
T-2561	you zo

S-5095	非常感谢 。 （ 鼓掌 ）
T-5095	thank you very much .
H-5095	-0.22272256016731262	thank you very much .
D-5095	-0.22272256016731262	thank you very much .
P-5095	-0.5661 -0.0719 -0.2169 -0.0356 -0.2466 -0.1992
S-4919	我 当时 非常 震惊 ，
T-4919	i was so shocked .
H-4919	-0.6475453972816467	and i was really shocked .
D-4919	-0.6475453972816467	and i was really shocked .
P-4919	-1.5954 -0.3167 -0.1746 -1.7065 -0.4474 -0.2023 -0.0899
S-4727	讲完 了 ， 谢谢 ！
T-4727	peace . thank you .
H-4727	-0.8000348806381226	all right , thank you .
D-4727	-0.8000348806381226	all right , thank you .
P-4727	-3.2135 -0.2476 -0.7908 -0.5235 -0.1213 -0.5732 -0.1303
S-4509	你 可以 是 非凡 的
T-4509	you can be extraordinary .
H-4509	-0.6255654096603394	you can be extraordinary .
D-4509	-0.6255654096603394	you can be extraordinary .
P-4509	-0.7961 -0.8040 -0.4726 -1.3203 -0.2453 -0.1151
S-4146	如今 我 22 岁 。
T-4146	today i am 22 .
H-4146	-0.49341267347335815	now i ' m 22 years old .
D-4146	-0.49341267347335815	now i ' m 22 years old .
P-4

...

S-3988	可是 ， 2011 年 的 今天 ， 如果 你 从 任何 一家 大型 的 激光打印机 厂商 买 一台 彩色 激光打印机 打印 出 一页 ， 这页 纸 最后 会 有 一些 小 黄点 印 在 每 一页 上 图案 让 这页 纸 是 独特 对 你 和 你 的 打印机 都 是 。
T-3988	but today in 2011 , if you go and buy a color laser printer from any major laser printer manufacturer and print a page , that page will end up having slight yellow dots printed on every single page in a pattern which makes the page unique to you and to your printer .
H-3988	-0.9831046462059021	but today , 2011 , if you buy a page from any big photovoltaic printer and print a page , the page will have little yellow slices of paper which are unique to you and your printer .
D-3988	-0.9831046462059021	but today , 2011 , if you buy a page from any big photovoltaic printer and print a page , the page will have little yellow slices of paper which are unique to you and your printer .
P-3988	-1.4991 -0.4399 -0.6043 -0.3111 -0.1053 -0.2551 -0.1393 -0.4574 -0.2271 -3.6263 -0.2393 -0.2701 -0.4600 -2.3332 -2.0087 -0.0322 -1.3205 -1.8867 -0.9792 -1.99

S-4006	这些 人 在线 挣 了 很多 钱 ， 但 他们 挣 的 钱 是 非法 的 通过 使用 类似 银行 木马 从 我们 账户 偷钱 当 我们 在 网上 进行 银行业务 ， 或者 使用 键盘记录 来 收集 我们 的 信用卡 信息 当 我们 通过 被 感染 的 电脑 来 在线 购物 的 时候 。
T-4006	these guys make their fortunes online , but they make it through the illegal means of using things like banking trojans to steal money from our bank accounts while we do online banking , or with keyloggers to collect our credit card information while we are doing online shopping from an infected computer .
H-4006	-0.9173539280891418	these people make a lot of money online , but they make a lot of money , and they make it illegal to steal from us from our account , when we ' re sending money online banking , or using a posting card to collect our credit card .
D-4006	-0.9173539280891418	these people make a lot of money online , but they make a lot of money , and they make it illegal to steal from us from our account , when we ' re sending money online banking , or using a posting card to collect our credit card .
P-4006	-1.1973 -0.

S-605	盖茨 ： 我们 还 没有 选定 一个 特定 的 地方 由于 到处 都 有 关于 核 问题 的 保密 规定 很多 人 表示 了 兴趣 俄罗斯 ， 印度 ， 中国 的 公司 都 表现 了 浓厚 的 兴趣 我 也 曾经 在 这里 找过 能源部长 谈论 如何 将 这件 事 融入 能源 日程表
T-605	bg : well , we have n 't picked a particular place , and there ' s all these interesting disclosure rules about anything that ' s called `` nuclear , ' ' so we ' ve got a lot of interest , that people from the company have been in russia , india , china -- i ' ve been back seeing the secretary of energy here , talking about how this fits into the energy agenda .
H-605	-0.9782946705818176	bill gates : we have n 't chosen a particular place , because there ' s a set of regulations all over the place , russia , russia , india , china , india , india , has been very interesting , and i ' ve been trying to find out how the minister of energy is going to be infiltrate this energy chart .
D-605	-0.9782946705818176	bill gates : we have n 't chosen a particular place , because there ' s a set of regulations all over the place , russia , russi

S-281	假设 我们 能 把 这种 观念 颠覆 把 网络 上下 颠覆 ， 相对 于 单线 信息 相互 链接 的 情况 ， 让 我们 开始 习惯 多样 信息 链接 多样 信息 ， 然后 看到 除 此 方式 外 无法 看到 的 ， 隐藏 规律 ？ 然后 看到 除 此 方式 外 无法 看到 的 ， 隐藏 规律 ？
T-281	what if we can actually turn that upside down and turn the web upside down , so that instead of navigating from one thing to the next , we get used to the habit of being able to go from many things to many things , and then being able to see the patterns that were otherwise hidden ?
H-281	-1.0668621063232422	and if we can turn this concept around and turn it over and over , rather than just a single line of information , let ' s start to deal with the diversity of information , and see how we ca n 't see the pattern out there , and see the patterns that are out there , and see the patterns that are out there that ca n 't be seen out there .
D-281	-1.0668621063232422	and if we can turn this concept around and turn it over and over , rather than just a single line of information , let ' s start to deal with the diversity of info

H-3729	-0.8764351606369019	i ' m speaking to you this morning with a great diver , and i ' d like to share with you -- i ' ve developed his technology called `` bows , ' ' some of you may have heard of it as a big room .
D-3729	-0.8764351606369019	i ' m speaking to you this morning with a great diver , and i ' d like to share with you -- i ' ve developed his technology called `` bows , ' ' some of you may have heard of it as a big room .
P-3729	-1.1176 -1.9459 -1.4233 -1.6275 -0.2712 -1.1284 -1.1258 -0.1131 -0.9251 -0.6897 -0.3973 -0.0215 -0.9989 -1.1537 -3.3277 -0.2214 -1.8034 -1.2551 -0.0626 -0.1303 -0.0433 -0.4104 -0.2182 -1.0946 -0.7403 -1.1406 -0.5306 -2.4659 -0.6063 -1.4256 -0.2671 -1.7686 -0.5059 -1.4039 -0.8473 -0.1924 -0.1112 -0.8318 -0.4103 -2.0198 -0.3827 -0.6081 -0.0454 -0.8902 -2.0311 -0.2177 -0.6019 -2.5008 -0.2150 -0.3290 -0.1029
S-2970	通过 匹配 不同 照片 边缘 的 色彩 ， 对比度 以及 亮度 再 加入 一下 摄影 瑕疵 比如 视野 的 深度 去 饱和 的 色彩 和 噪音 我们 可以 除去 不同 照片 之间 的 边界 使 其 看起来 就 像是 一张 单一 的 照片 即使 一张 照片 可 包含 上 百

# Tools

In [7]:
!ls ../data/IWSLT15/zh_en/model/20210107context/checkpoints

In [3]:
!nvidia-smi

Wed Jan  6 01:20:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     Off  | 00000000:01:00.0 Off |                  Off |
| 65%   82C    P2   265W / 260W |  33980MiB / 48598MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!kill 4975